# Initial State Configuration for Processor Sharing Queue

This example demonstrates initial state configuration for a multi-class closed network with Processor Sharing (PS) scheduling and class switching dynamics.

In [ ]:
// Kotlin notebook
import jline.*
import jline.lang.*
import jline.lang.nodes.*
import jline.lang.processes.*
import jline.lang.constant.*
import jline.solvers.ctmc.*
import jline.solvers.jmt.*
import jline.solvers.ssa.*
import jline.solvers.fluid.*
import jline.solvers.mva.*
import jline.solvers.nc.*
import jline.util.matrix.*

In [ ]:
val model = Network("model")

// Create nodes
val infiniteServer = Delay(model, "InfiniteServer")
val queue1 = Queue(model, "Queue1", SchedStrategy.PS)  // Processor Sharing queue
queue1.setNumberOfServers(2)  // 2-server PS queue

// Create two closed classes with different populations
val class1 = ClosedClass(model, "Class1", 3, infiniteServer, 0)  // 3 jobs of Class1 start at InfiniteServer
val class2 = ClosedClass(model, "Class2", 1, infiniteServer, 0)  // 1 job of Class2 starts at InfiniteServer

println("Model Setup:")
println("- Class 1: ${class1.population} jobs")
println("- Class 2: ${class2.population} jobs")
println("- Total jobs: ${class1.population + class2.population}")
println("- Queue1 scheduling: Processor Sharing")
println("- Queue1 servers: ${queue1.numberOfServers}")
println("- Initial state: All jobs start at InfiniteServer")

In [ ]:
// Set service processes with different rates
infiniteServer.setService(class1, Exp(3.0))  // Class1: Fast service at infinite server
infiniteServer.setService(class2, Exp(0.5))  // Class2: Slow service at infinite server

queue1.setService(class1, Exp(0.1))  // Class1: Very slow service at PS queue
queue1.setService(class2, Exp(1.0))  // Class2: Moderate service at PS queue

println("\nService Configurations:")
println("InfiniteServer (Delay):")
println("  - Class1: Exponential(μ=3.0) - Fast service")
println("  - Class2: Exponential(μ=0.5) - Slow service")
println("Queue1 (Processor Sharing):")
println("  - Class1: Exponential(μ=0.1) - Very slow service")
println("  - Class2: Exponential(μ=1.0) - Moderate service")
println("\nService Rate Ratios:")
println("  - Class1: InfiniteServer 30x faster than Queue1")
println("  - Class2: Queue1 2x faster than InfiniteServer")

In [ ]:
// Set up complex routing with class switching
val P = model.initRoutingMatrix()

// Class 1 to Class 1 routing
P.set(class1, class1, infiniteServer, infiniteServer, 0.3)  // Stay at InfiniteServer (30%)
P.set(class1, class1, infiniteServer, queue1, 0.1)         // Move to Queue1 as Class1 (10%)
P.set(class1, class1, queue1, infiniteServer, 0.2)         // Return to InfiniteServer (20%)
P.set(class1, class1, queue1, queue1, 0.0)                 // Cannot stay at Queue1

// Class 1 to Class 2 routing (class switching)
P.set(class1, class2, infiniteServer, infiniteServer, 0.6)  // Switch to Class2 at InfiniteServer (60%)
P.set(class1, class2, infiniteServer, queue1, 0.0)
P.set(class1, class2, queue1, infiniteServer, 0.8)         // Switch to Class2 and return (80%)
P.set(class1, class2, queue1, queue1, 0.0)

// Class 2 to Class 2 routing
P.set(class2, class2, infiniteServer, infiniteServer, 0.0)
P.set(class2, class2, infiniteServer, queue1, 1.0)         // Always move to Queue1 (100%)
P.set(class2, class2, queue1, infiniteServer, 0.0)
P.set(class2, class2, queue1, queue1, 0.0)

// Class 2 to Class 1 routing (class switching back)
P.set(class2, class1, infiniteServer, infiniteServer, 0.0)
P.set(class2, class1, infiniteServer, queue1, 0.0)
P.set(class2, class1, queue1, infiniteServer, 1.0)         // Switch to Class1 and return (100%)
P.set(class2, class1, queue1, queue1, 0.0)

model.link(P)

println("\nRouting Configuration:")
println("Class1 routing:")
println("  - 30% stay at InfiniteServer")
println("  - 10% move to Queue1 (as Class1)")
println("  - 60% switch to Class2 at InfiniteServer")
println("  - From Queue1: 20% return as Class1, 80% switch to Class2")
println("Class2 routing:")
println("  - Always moves InfiniteServer→Queue1")
println("  - Always switches to Class1 and returns Queue1→InfiniteServer")

In [ ]:
// Solve with multiple solvers to compare results
val results = mutableMapOf<String, Matrix>()

// Try CTMC solver
try {
    val solverCtmc = CTMC(model)
    results["CTMC"] = solverCtmc.avgTable
    println("\n=== CTMC Results ===")
    results["CTMC"]!!.print()
} catch (e: Exception) {
    println("\nCTMC solver failed: ${e.message}")
}

In [ ]:
// Try JMT simulation
try {
    val jmtOptions = JMT.defaultOptions()
    jmtOptions.samples = 100000
    jmtOptions.seed = 23000
    
    val solverJmt = JMT(model, jmtOptions)
    results["JMT"] = solverJmt.avgTable
    println("\n=== JMT Simulation Results ===")
    results["JMT"]!!.print()
} catch (e: Exception) {
    println("\nJMT solver failed: ${e.message}")
}

In [ ]:
// Try SSA solver
try {
    val ssaOptions = SSA.defaultOptions()
    ssaOptions.samples = 100000
    ssaOptions.seed = 23000
    
    val solverSsa = SSA(model, ssaOptions)
    results["SSA"] = solverSsa.avgTable
    println("\n=== SSA Results ===")
    results["SSA"]!!.print()
} catch (e: Exception) {
    println("\nSSA solver failed: ${e.message}")
}

In [ ]:
// Try Fluid solver
try {
    val solverFluid = FLD(model)
    results["Fluid"] = solverFluid.avgTable
    println("\n=== Fluid Results ===")
    results["Fluid"]!!.print()
} catch (e: Exception) {
    println("\nFluid solver failed: ${e.message}")
}

In [ ]:
// Try MVA solver
try {
    val solverMva = MVA(model)
    results["MVA"] = solverMva.avgTable
    println("\n=== MVA Results ===")
    results["MVA"]!!.print()
} catch (e: Exception) {
    println("\nMVA solver failed: ${e.message}")
}

In [ ]:
// Try NC solver
try {
    val solverNc = NC(model)
    results["NC"] = solverNc.avgTable
    println("\n=== NC Results ===")
    results["NC"]!!.print()
} catch (e: Exception) {
    println("\nNC solver failed: ${e.message}")
}

In [ ]:
// Analyze the processor sharing system
println("\n=== Processor Sharing System Analysis ===")
println("This model demonstrates several advanced queueing features:")
println("\n1. Processor Sharing (PS) Scheduling:")
println("   - All jobs at Queue1 receive equal share of service capacity")
println("   - No job blocking; all jobs make progress simultaneously")
println("   - Service rate per job decreases with queue length")
println("\n2. Initial State Configuration:")
println("   - All 4 jobs start at InfiniteServer (no congestion initially)")
println("   - System evolution depends on routing and class switching")
println("\n3. Asymmetric Service Rates:")
println("   - Class1 prefers InfiniteServer (3.0 vs 0.1 rate)")
println("   - Class2 prefers Queue1 (1.0 vs 0.5 rate)")
println("   - Creates natural class segregation tendencies")
println("\n4. Complex Class Switching:")
println("   - Class1→Class2 switching happens primarily at InfiniteServer")
println("   - Class2→Class1 switching happens at Queue1")
println("   - Creates dynamic class population changes")
println("\n5. Multi-Server PS Queue:")
println("   - 2 servers provide additional capacity")
println("   - PS scheduling distributes load evenly")
println("   - Reduces impact of slow Class1 service at Queue1")
println("\nSolver Comparison:")
if (results.isNotEmpty()) {
    println("   - Successfully solved with: ${results.keys.joinToString(", ")}")
    println("   - Different solvers may show varying accuracy for PS systems")
    println("   - CTMC provides exact results for smaller state spaces")
    println("   - Simulation methods handle complex routing well")
} else {
    println("   - Complex routing and PS scheduling challenged the solvers")
    println("   - Some solvers may not support all features")
}
println("\nThis model is relevant for:")
println("- Time-sharing computer systems")
println("- Bandwidth allocation in networks")
println("- Fair resource sharing systems")
println("- Systems with dynamic job type changes")

This example demonstrates:

1. **Processor Sharing Discipline**: 
   - Jobs share service capacity equally at Queue1
   - No job blocking - all jobs make simultaneous progress
   - Service rate per job is inversely proportional to queue length
2. **Complex Class Dynamics**:
   - Two classes with very different service preferences
   - Class1 favors InfiniteServer (30x faster service)
   - Class2 favors Queue1 (2x faster service)
3. **Dynamic Class Switching**:
   - Probabilistic transitions between job classes
   - Different switching probabilities at different stations
   - Creates time-varying class populations
4. **Initial State Impact**:
   - All jobs start at InfiniteServer (balanced initial state)
   - Evolution depends on routing probabilities and service rates
   - Different from previous examples with congested initial states
5. **Solver Validation**:
   - Tests multiple solution approaches
   - Validates consistency across different methods
   - Some solvers may be better suited for PS systems

This model is particularly relevant for:
- Operating systems with time-sharing
- Network bandwidth allocation
- Cloud computing resource management
- Any system requiring fair resource sharing among competing processes